# Importance Sampling for a European Call

The goal of this exercise is to show how importance sampling can be used to provide a more accurate predictive interval for the price of an option.

The value of a European call option, $ V(t, S_t) $ is defined as $ \mathbb{E}^\mathbb{Q} \left[ e^{-r(T-t)}\max(S_T - K, 0) \right | \mathcal{F}_t] $

That is, the risk neutral expectation of the discounted terminal payoff conditional on all information obtained until the current time.

Consider the following scenario:

$S(0) = 50, \sigma = 0.2, r = 0.06$, and $T = 0.5$

Let us consider a range of strikes $K$ with values 40, 50, 70, and 90.

Intuitively, we expect standard monte carlo to perform well for strikes close to the starting value, 40 and 50 in this case, and worse when the strikes are far from the current option value, in this example 70 and 90. Let's see what happens.

First, as a reference, let's use the black scholes equations to compute the theoretical values as a reference.

In [2]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

import BMS


#Define parameters
T = 0.5
r = 0.06
sigma = 0.2
S0 = 50
Kn = np.array([40,50,70,90])
MC = 10000 #number of monte carlo iterations.
BS_exact = [0 for _ in range(len(Kn))]

for i in range(0, len(Kn)):
    BS_exact[i] = BMS.BMS_price('call', S = S0, K = Kn[i], r = r, q = 0, sigma = sigma, T = T, t = 0)

#Format the output to 4 decimal points
print([ '%.4f' % elem for elem in BS_exact ])

['11.2732', '3.5779', '0.0441', '0.0001']


Next, let's proceed with the standard Monte Carlo simulation

In [5]:

#Allocate space for storage
MCcall = [0 for _ in range(len(Kn))]
CIWidth_MC = [0 for _ in range(len(Kn))]
CI_MCcall = np.zeros(shape = (len(Kn), 2))

#set seed
np.random.seed(seed=233423)

#Sample from the normal distribution
Z = scipy.stats.norm.rvs(loc = 0, scale = 1, size = MC)
# Use standard MC method to estimate the option price
#This is a Geoemtric brownian motion construction
ZForMC = (r - sigma**2 / 2) * T + sigma * np.sqrt(T) * Z;
SFinal = S0 * np.exp(ZForMC);
for i in range(0, len(Kn)):
    K = Kn[i]
    #REMEMBER TO USE numpy.maximum, not np.max. They are two different functions!
    DiscPayoff = np.exp(-r * T) * np.maximum(SFinal - K, 0)
    MCcall[i] = np.mean(DiscPayoff)
    StdDev = np.std(DiscPayoff)
    #Confidence Interval
    CI_MCcall[i,0] = MCcall[i] - 1.96*StdDev/np.sqrt(MC)
    CI_MCcall[i,1] = MCcall[i] + 1.96*StdDev/np.sqrt(MC)
    CIWidth_MC[i] = 1.96 * StdDev / np.sqrt(MC);

print([ '%.4f' % elem for elem in MCcall ])


['11.3164', '3.6071', '0.0556', '0.0000']


Visually, the standard Monte Carlo seems fairly accurate; however, on a percentage basis, we notice that the deviations for the latter 2 strikes seem to be very high.

Now let's see what kind of improvements we can get with importance sampling.

First, in order to perform importance sampling, we need to pick a probability density function to use in order to construct our likelihood ratio density. We want to choose a density that helps us sample from the important region. What is the important region? Intuitively, given that we are analyzing a range of strikes, and given that an option changes from valuable to having no value depending on which side of the strike it's on, we want to focus on what happens when the process $S_t$ is at or near the strike value.

Thus, we want to find to choose an importance sampling distribution so that $ \mathbb{E} \left[S_T \right] = K $

Since we model the process $S_t$ is a Geometric Brownian Motion, which as lognormal density, we can know  $ \mathbb{E} \left[S_T \right] = S(0)e^{\mu T} = K $

Every quantitiy is known here except for $\mu$, so solving for it we have $\mu = \frac{1}{T} \log \left( \frac{K}{S(0)} \right) $

Thus, we drift the process depending on what strike value we are investigating, which makes sense, because the further away the strike is from the intial value of our process, the greater a drift the process must have in order for the probability to be high that we "see" those values.

Now, for some math, We want to estimate $\alpha$ which we define as $\alpha = \mathbb{E}\left[ h(X) \right] $ where h is our payoff function. This can be rewritten as

$$\alpha = \mathbb{E}\left[ e^{-rT}[S(T) - K]^+ \right] $$

If we simulate values of $S$, this is sufficient, but we can make one more simplification to make the code a bit simpler:
$$\alpha = \mathbb{E}\left[ e^{-rT}[S(0)e^X - K]^+ \right] $$

where $X \stackrel{}{\sim}  \mathcal{N}((r - \sigma^2/2)T, \sigma^2T) $

In [35]:

#Allocate space for storage
IScall = [0 for _ in range(len(Kn))]
CIWidth_IS = [0 for _ in range(len(Kn))]
CI_IScall = np.zeros(shape = (len(Kn), 2))

for i in range(0, len(Kn)-1):
    K = Kn[i]

    mu = (1/T)*np.log(K/S0);
    #Shift the normal random variables
    ZForIS = (mu-sigma**2/2) * T + sigma*np.sqrt(T)*Z
    ISSFinal = S0 * np.exp(ZForIS);
    ISDiscPayoff = np.exp(-r * T) * np.maximum(ISSFinal - K, 0)

    #x1 represents the drifted values taken from the original distribution
    #x2 represents the drifted values taken from the drifted distribution
    #dividing gives us the likelihood ratio
    x1 = scipy.stats.norm.pdf(x = ZForIS, loc = (r - sigma**2/2)*T, scale = sigma * np.sqrt(T)) 
    x2 = scipy.stats.norm.pdf(x = ZForIS, loc = (mu - sigma**2/2)*T, scale = sigma * np.sqrt(T))

    #LR = np.divide ( x1 = x1, x2 = x2) 
    LR = x1 / x2
    ISValues = np.multiply(ISDiscPayoff , LR)
    IScall[i] = np.mean(ISValues)
    StdDev = np.std(ISValues)
    CI_IScall[i,:] = [IScall[i] - 1.96*StdDev/np.sqrt(MC), IScall[i] + 1.96*StdDev/np.sqrt(MC)]
    CIWidth_IS[i] = 1.96 * StdDev / np.sqrt(MC)

print([ '%.4f' % elem for elem in IScall ])

['11.9689', '3.6548', '0.0432', '0.0000']


We can see for the last two values especially we see an improvement on the point estimators. If we look at confidence intervals and compute variance reduction ratios, it's even more clear. The full code, all done in one loop, is below. This is necessary because the variance reduction ratios are computed on each iteration. The only way to do this without looping through all at once would be to store every value from each individual simulation, which while possible is generally unneeded.

In [43]:


#Allocate space for storage
MCcall = [0 for _ in range(len(Kn))]
CIWidth_MC = [0 for _ in range(len(Kn))]
CI_MCcall = np.zeros(shape = (len(Kn), 2))

IScall = [0 for _ in range(len(Kn))]
CIWidth_IS = [0 for _ in range(len(Kn))]
CI_IScall = np.zeros(shape = (len(Kn), 2))

variance_reduction = [0 for _ in range(len(Kn))]


#set seed
np.random.seed(seed=233423)

#Sample from the normal distribution
Z = scipy.stats.norm.rvs(loc = 0, scale = 1, size = MC)
# Use standard MC method to estimate the option price
#This is a Geoemtric brownian motion construction
ZForMC = (r - sigma**2 / 2) * T + sigma * np.sqrt(T) * Z;
SFinal = S0 * np.exp(ZForMC);
for i in range(0, len(Kn)):
    K = Kn[i]
    ## Standard Monte Carlo
    #REMEMBER TO USE numpy.maximum, not np.max. They are two different functions!
    DiscPayoff = np.exp(-r * T) * np.maximum(SFinal - K, 0)
    MCcall[i] = np.mean(DiscPayoff)
    StdDev = np.std(DiscPayoff)
    #Confidence Interval
    CI_MCcall[i,0] = MCcall[i] - 1.96*StdDev/np.sqrt(MC)
    CI_MCcall[i,1] = MCcall[i] + 1.96*StdDev/np.sqrt(MC)
    CIWidth_MC[i] = 1.96 * StdDev / np.sqrt(MC);

    #Importance Sampling

    mu = (1/T)*np.log(K/S0);
    #Shift the normal random variables
    ZForIS = (mu-sigma**2/2) * T + sigma*np.sqrt(T)*Z
    ISSFinal = S0 * np.exp(ZForIS);
    ISDiscPayoff = np.exp(-r * T) * np.maximum(ISSFinal - K, 0)

    #x1 represents the drifted values taken from the original distribution
    #x2 represents the drifted values taken from the drifted distribution
    #dividing gives us the likelihood ratio
    x1 = scipy.stats.norm.pdf(x = ZForIS, loc = (r - sigma**2/2)*T, scale = sigma * np.sqrt(T)) 
    x2 = scipy.stats.norm.pdf(x = ZForIS, loc = (mu - sigma**2/2)*T, scale = sigma * np.sqrt(T))

    #LR = np.divide ( x1 = x1, x2 = x2) 
    LR = x1 / x2
    ISValues = np.multiply(ISDiscPayoff , LR)
    IScall[i] = np.mean(ISValues)
    StdDev = np.std(ISValues)
    CI_IScall[i,:] = [IScall[i] - 1.96*StdDev/np.sqrt(MC), IScall[i] + 1.96*StdDev/np.sqrt(MC)]
    CIWidth_IS[i] = 1.96 * StdDev / np.sqrt(MC)

    #Variance Reduction from MC to IS
    variance_reduction[i] = (np.std(DiscPayoff)/np.std(ISValues))**2

print("Monte Carlo Point Estimators")
print([ '%.4f' % elem for elem in MCcall ])
print("Monte Carlo Confidence Intervals")
print(CI_MCcall)

print("Importance Sampling Point Estimators")
print([ '%.4f' % elem for elem in IScall ])
print("Importance Sampling Confidence Intervals")
print(CI_IScall)

print("Variance Reduction Ratios")
print([ '%.4f' % elem for elem in variance_reduction ])


Monte Carlo Point Estimators
['11.3164', '3.6071', '0.0556', '0.0000']
Monte Carlo Confidence Intervals
[[11.17875058 11.45414765]
 [ 3.50712848  3.70698358]
 [ 0.04447797  0.06679917]
 [ 0.          0.        ]]
Importance Sampling Point Estimators
['11.9689', '3.6548', '0.0432', '0.0001']
Importance Sampling Confidence Intervals
[[1.04594278e+01 1.34784538e+01]
 [3.52332058e+00 3.78635127e+00]
 [4.21820526e-02 4.42788671e-02]
 [8.16526086e-05 8.66027566e-05]]
Variance Reduction Ratios
['0.0083', '0.5773', '113.3222', '0.0000']


Note that the variance reduction ratio is invalid for the last case. We can clearly see an appreciable improvement for the strike at 70.